# 서론

&emsp;이번에 LLM과 RAG 기술을 활용해 사용자 질문에 답변하는 OpenAI의 gpt-4o 모델을 기반으로 하는 챗봇을 만들게 되었습니다. 데이터 리스트로는 초거대 언어모델 연구 동향이라는 논문을 사용했습니다. 이번 챗봇을 구축하면서 LLM과 RAG에 배운 내용을 복습하였으며, 배운 내용에서 pdf 형식의 문서를 불러와 RAG을 구축하는 걸 목표로 했습니다.   
&emsp;데이터 리스트로 쓰인 초거대 언어모델 연구 동향은 LLM 요소에 익숙해지기 좋은 자료로 LLM에 업데이트 되어있지 않은 최신 인공지능 연구동향에 대해 정확히 답변할 수 있는 챗봇을 만들 수 있었습니다.

# 목차

0. **필요한 모듈 불러오기**
1. **사용환경 준비**
2. **모델 로드하기**
3. **문서 로드하기**
4. **문서 청크로 나누기**
   1. **CharacterTextSplitter**
   2. **RecursiveCharacterTextSplitter**
5. **벡터 임베딩 생성**
6. **벡터 스토어 생성**
7. **FAISS를 Retriever로 변환**
8. **프롬프트 템플릿 정의**
9. **RAG 체인 구성**
10. **챗봇 구동 확인**

# 0. 필요한 모듈 불러오기
&emsp;코드에 사용된 각 모듈과 라이브러리의 역할을 설명하며, 전체적인 흐름도 함께 안내합니다.

In [8]:
import os
from getpass import getpass
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

- `os`:   
&emsp;Python의 기본 모듈인 os는 파일 및 디렉터리와 관련된 다양한 작업을 수행할 수 있게 해줍니다. 예를 들어, 환경 변수 접근, 파일 경로 조작 등이 가능합니다.
- `getpass`:   
&emsp;getpass는 사용자로부터 안전하게 비밀번호를 입력받기 위한 모듈입니다. 터미널에 입력된 비밀번호를 표시하지 않고 읽을 수 있게 해줍니다. 이를 통해 OpenAI API 키와 같은 민감한 정보를 안전하게 입력받을 수 있습니다.
- `ChatOpenAI`:   
&emsp;OpenAI의 GPT 모델을 기반으로 동작하는 ChatOpenAI 클래스는 LangChain에서 대화형 AI를 구현하기 위해 사용됩니다. 이 클래스는 OpenAI의 대화형 API와 통합되어 텍스트 생성을 지원합니다.
- `HumanMessage`:   
&emsp;LangChain의 메시지 모델 중 하나로, 인간이 입력한 메시지를 표현합니다. 대화에서 사용자의 발화를 모델에 전달하는 데 사용됩니다.
- `PyPDFLoader`:   
&emsp;PDF 파일에서 텍스트 데이터를 추출하기 위한 모듈입니다. PDF 문서를 로드하고 내부 텍스트를 분석해 LangChain 워크플로우에서 사용할 수 있게 합니다.
- `CharacterTextSplitter`:   
&emsp;텍스트 데이터를 특정 문자 단위로 나누는 단순한 텍스트 분할기입니다. 텍스트를 처리하기 쉽게 조각으로 나누는 역할을 합니다.
- `RecursiveCharacterTextSplitter`:   
&emsp;CharacterTextSplitter의 확장 버전으로, 텍스트를 더 정교하게 분할합니다. 긴 텍스트를 처리하는 데 유용하며, 특정 기준을 기반으로 텍스트를 분할합니다.
- `OpenAIEmbeddings`:   
&emsp;텍스트 데이터를 벡터 형식으로 변환하여 의미적 비교가 가능하도록 만들어주는 모듈입니다. OpenAI API를 사용해 임베딩을 생성합니다.
- `FAISS`:   
&emsp;Facebook AI에서 개발한 빠른 벡터 검색 라이브러리입니다. 임베딩된 데이터를 효율적으로 검색하고 유사도를 계산하는 데 사용됩니다. FAISS는 검색과 분류 작업에서 특히 유용합니다.
- `ChatPromptTemplate`:   
&emsp;사용자와 AI 간의 대화를 정의하기 위한 템플릿입니다. LangChain에서 대화 흐름과 프롬프트 형식을 제어하는 데 사용됩니다.
- `LLMChain`:   
&emsp;LangChain의 핵심 구성 요소 중 하나로, 대규모 언어 모델(LLM)과의 상호작용을 체계화합니다. 프롬프트, 입력 데이터, 출력 데이터를 연결하여 워크플로우를 구성합니다.

# 1. 사용환경 준비
&emsp;코드에 사용된 각 모듈과 라이브러리의 역할을 설명하며, 전체적인 흐름도 함께 안내합니다.

In [10]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API key 입력: ")

OpenAI API key 입력:  ········


&emsp;이 코드는 주로 LangChain을 사용하여 PDF 파일을 분석하고, 임베딩을 생성하며, 대화형 AI 워크플로우를 구축하는 데 사용됩니다. 이러한 구성 요소들은 대규모 언어 모델을 활용한 다양한 작업을 자동화하고 체계화하는 데 중요한 역할을 합니다.

# 2. 모델 로드하기
- 이 코드는 **LangChain** 라이브러리의 `ChatOpenAI` 클래스를 사용하여 **GPT-4 모델**의 인스턴스를 생성하는 코드입니다.

In [12]:
# 모델 초기화
model = ChatOpenAI(model="gpt-4")

- `ChatOpenAI` 클래스: LangChain에서 OpenAI GPT 모델을 사용하는 인터페이스 제공하는 클래스입니다.
- `model="gpt-4"`: GPT-4 모델을 지정하여 고성능 텍스트 생성 및 처리가 가능합니다.
- 이 코드로 생성된 `model` 객체는 LangChain에서 텍스트 생성과 같은 다양한 자연어 처리 작업에 활용됩니다.

# 3. 문서 로드하기
&emsp;이 코드는 **PDF 파일에서 텍스트 데이터를 읽어와 처리**하기 위해 LangChain의 `PyPDFLoader` 클래스를 사용하는 코드입니다.

In [14]:
# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("LLM_movement.pdf")

# 페이지 별 문서 로드
docs = loader.load()

- `PyPDFLoader`는 PDF 파일에서 텍스트를 추출하고 문서 객체 리스트로 반환하는 LangChain의 강력한 도구입니다.
- `loader.load()`는 페이지별 텍스트와 메타데이터를 포함한 문서 리스트를 생성하며, 추출된 데이터는 텍스트 분할, 검색, 요약 등 다양한 작업에 활용됩니다.

# 4. 문서 청크로 나누기
&emsp;문서 청킹하는 방법으로는 CharacterTextSplitter, RecursiveCharacterTextSplitter 이 두 가지가 있습니다. 저는 CharacterTextSplitter 방법으로 문서를 청킹하였습니다.

## A. CharacterTextSplitter
&emsp;이 코드는 **LangChain의 텍스트 분할기**를 사용하여 PDF 문서에서 추출된 텍스트 데이터를 <u>작은 조각(chunk)</u>으로 나누는 작업을 수행합니다.

In [16]:
text_splitter = CharacterTextSplitter(separator="\n\n", chunk_size=100, chunk_overlap=10, length_function=len, is_separator_regex=False)

splits = text_splitter.split_documents(docs)
splits[:10] # 청킹된 내용을 상위 10개까지 출력

[Document(metadata={'source': 'LLM_movement.pdf', 'page': 0}, page_content='8 특집원고  초거대 언어모델 연구 동향\n초거대 언어모델 연구 동향\n업스테이지  박찬준*･이원성･김윤기･김지후･이활석\n \n1. 서  론1)\nChatGPT1)와 같은 초거대 언어모델(Large Language \nModel, LLM) 의 등장으로 기존에 병렬적으로 연구되\n던 다양한 자연언어처리 하위 분야들이 하나의 모델\n로 처리되고 있으며, 태스크 수렴 현상 (Converge)이 \n발생하고 있다. 즉 하나의 LLM으로 번역, 요약, 질의\n응답, 형태소분석 등의 작업을 모두 처리할 수 있게 \n되었다. 프롬프트 (Prompt)를 어떻게 모델에게 입력하\n느냐에 따라서 LLM의 다양한 능력들이 창발되고, 이\n에 따라 사용자의 목적에 맞는 출력을 생성하는 패러\n다임을 맞이하게 되었다 [1].\nLLM은 최근 몇 년 간의 연구 동향에 따라 뛰어난 \n발전을 이루고 있다. 이러한 발전은 몇 가지 주요한 \n요인에 기반하고 있으며, 이 요인들은 현대 자연언어\n처리 (Natural Language Processing, NLP) 연구의 핵심\n적인 추세로 간주된다. 첫째로, 데이터의 양적 확대는 \n무시할 수 없는 중요한 요인이다. 디지털화의 선도로, \n텍스트 데이터의 양이 기하급수적으로 증가하였고, \n이는 연구의 질적 변화를 가져왔다. 대규모 코퍼스의 \n활용은 LLM의 일반화 능력을 향상시키며, 다양한 맥\n락과 주제에 대한 깊은 학습을 가능하게 한다. 둘째\n로, 컴퓨팅 기술의 진보는 LLM의 발전에 있어 결정\n적이었다. 특히, Graphics Processing Unit (GPU) 및 \nTensor Processing Unit (TPU) 와 같은 고성능 병렬 처\n리 하드웨어의 개발은 모델 학습에 있어 병목 현상을 \n크게 완화시켰다. 이로 인해 연구자들은 모델의 복잡\n성을 키우고, 더욱 깊은 신경망 구조

**초기화 시 전달되는 매개변수**
1. `separator="\n\n"`
- 텍스트를 분할할 때 기준이 되는 **구분자**(separator)를 지정합니다.
- `"\n\n"`: 두 줄 간격으로 나눕니다. 텍스트가 문단 단위로 분리될 가능성이 높습니다.
- 이 값을 변경하면 다른 기준(예: 마침표 `"."`, 쉼표 `","` 등)으로 텍스트를 분할할 수도 있습니다.

2. `chunk_size=100`
- 각 텍스트 조각의 최대 길이를 지정합니다.
- 여기서는 각 텍스트 조각이 최대 **100개의 문자**를 포함하도록 설정됩니다.
- 너무 길면 처리 효율이 떨어지고, 너무 짧으면 문맥 정보가 부족해질 수 있습니다.

3. `chunk_overlap=10`
- 각 조각 사이에 겹치는 문자의 수를 지정합니다.
- 여기서는 조각 간에 **10개의 문자**가 겹칩니다.
- 겹침이 있으면 문맥 연결이 더 자연스러워져 문서 내용의 연속성을 유지할 수 있습니다.

4. `length_function=len`
- 각 텍스트의 길이를 계산하는 함수입니다.
- 기본값은 `len`, 즉 문자의 개수를 기준으로 길이를 측정합니다.
- 특정 요구사항에 따라 단어 수를 기준으로 하는 함수 등을 사용할 수도 있습니다.

5. `is_separator_regex=False`
- `separator`가 정규 표현식인지 여부를 나타냅니다.
- `False`로 설정되어 있으므로, 여기서는 `"\n\n"`을 문자 그대로 사용합니다.

## B. RecursiveCharacterTextSplitter
&emsp;이 코드는 LangChain의 **RecursiveCharacterTextSplitter**를 사용하여 PDF 문서에서 추출된 텍스트를 <u>계층적으로 텍스트 조각(chunk)</u>으로 나누는 작업을 수행합니다.

In [18]:
recursive_text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10, length_function=len, is_separator_regex=False)

splits = recursive_text_splitter.split_documents(docs)
splits[:10] # 청킹된 내용을 상위 10개까지 출력

[Document(metadata={'source': 'LLM_movement.pdf', 'page': 0}, page_content='8 특집원고  초거대 언어모델 연구 동향\n초거대 언어모델 연구 동향\n업스테이지  박찬준*･이원성･김윤기･김지후･이활석\n \n1. 서  론1)'),
 Document(metadata={'source': 'LLM_movement.pdf', 'page': 0}, page_content='1. 서  론1)\nChatGPT1)와 같은 초거대 언어모델(Large Language \nModel, LLM) 의 등장으로 기존에 병렬적으로 연구되'),
 Document(metadata={'source': 'LLM_movement.pdf', 'page': 0}, page_content='던 다양한 자연언어처리 하위 분야들이 하나의 모델\n로 처리되고 있으며, 태스크 수렴 현상 (Converge)이 \n발생하고 있다. 즉 하나의 LLM으로 번역, 요약, 질의'),
 Document(metadata={'source': 'LLM_movement.pdf', 'page': 0}, page_content='응답, 형태소분석 등의 작업을 모두 처리할 수 있게 \n되었다. 프롬프트 (Prompt)를 어떻게 모델에게 입력하\n느냐에 따라서 LLM의 다양한 능력들이 창발되고, 이'),
 Document(metadata={'source': 'LLM_movement.pdf', 'page': 0}, page_content='에 따라 사용자의 목적에 맞는 출력을 생성하는 패러\n다임을 맞이하게 되었다 [1].\nLLM은 최근 몇 년 간의 연구 동향에 따라 뛰어난'),
 Document(metadata={'source': 'LLM_movement.pdf', 'page': 0}, page_content='발전을 이루고 있다. 이러한 발전은 몇 가지 주요한 \n요인에 기반하고 있으며, 이 요인들은 현대 자연언어'),
 Document(metadata={'source': 

&emsp;**초기화 시 전달되는 매개변수**
1. `chunk_size=100`
- 각 조각(chunk)의 최대 길이를 설정합니다.
- 여기서는 **100개의 문자**를 넘지 않도록 제한합니다.
2. `chunk_overlap=10`
- 조각 간에 **10개의 문자**를 겹치도록 설정합니다.
- 겹침을 통해 조각 간 문맥 연결성을 유지합니다.
3. `length_function=len`
- 조각의 길이를 계산하는 데 사용하는 함수입니다.
- 기본적으로 Python의 `len()` 함수로, 문자의 개수를 기준으로 길이를 측정합니다.
4. `is_separator_regex=False`
- 구분자가 정규 표현식인지 여부를 나타냅니다.
- `False`로 설정되었으므로, 구분자는 단순한 문자열로 취급됩니다.

# 5. 벡터 임베딩 생성
&emsp;이 코드는 OpenAI에서 제공하는 **임베딩 모델**을 초기화하여 텍스트를 벡터로 변환하기 위한 객체를 생성합니다. 벡터는 텍스트의 의미를 수치적으로 표현한 고차원 공간의 점으로, 검색, 분류, 추천 등 다양한 NLP(Natural Language Processing) 작업에서 활용됩니다.

In [20]:
# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

- `OpenAIEmbeddings` 객체는 OpenAI의 임베딩 API를 호출하여 텍스트를 고차원 벡터로 변환하는 데 사용됩니다.
- `"text-embedding-3-large"`는 정교한 텍스트 이해를 위한 대규모 임베딩 모델로, 유사성 계산, 검색, 분류 작업 등에 유용합니다.
- 이 코드는 임베딩 생성의 초기화 단계이며, 이후 텍스트 데이터를 실제로 벡터화하여 NLP 작업에 활용할 수 있습니다.

# 6. 벡터 스토어 생성
&emsp;이 코드는 LangChain에서 제공하는 <u>FAISS(벡터 검색 라이브러리)</u>를 사용하여 주어진 문서를 벡터화하고, 이를 기반으로 검색 가능한 <u>벡터 스토어(vector store)</u>를 생성하는 작업을 수행합니다.

In [22]:
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

- 이 코드는 **문서를 벡터화하고 검색 가능한 데이터베이스를 생성**하는 작업입니다.
- FAISS는 효율적인 벡터 검색을 제공하며, 텍스트 유사도 기반 검색과 질의응답 시스템에서 중요한 역할을 합니다.
- `vectorstore` 객체를 사용해 텍스트 검색, 추천, 클러스터링 등의 작업을 수행할 수 있습니다.

# 7. FAISS를 Retriever로 변환
&emsp;이 코드는 `vectorstore` 객체를 기반으로 **질의응답(retrieval)** 작업을 수행할 수 있는 <u>retriever(검색기)</u>를 생성하는 작업입니다.   
&emsp;사용자가 입력한 질의(query)에 대해 **유사성 검색**을 수행하여 가장 적합한 결과를 반환할 수 있도록 구성합니다.

In [24]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 1})

- 이 코드는 `vectorstore`에서 유사성 검색을 수행할 수 있는 **retriever** 객체를 생성합니다.
- 검색기는 사용자가 입력한 질의를 기반으로 가장 유사한 데이터를 반환합니다.
- `search_type="similarity"`와 `search_kwargs={"k": 1}` 설정을 통해 유사성 검색 방식과 반환 결과 개수를 제어합니다.
- 이 작업은 질의응답 시스템, 검색 엔진, 추천 시스템 등 다양한 NLP 응용 프로그램에서 유용하게 활용됩니다.

# 8. 프롬프트 템플릿 정의
&emsp;이 코드는 LangChain에서 `ChatPromptTemplate`을 사용해 **대화형 프롬프트 템플릿**을 정의하는 작업입니다.   
&emsp;주어진 <u>문맥(context)</u>에 기반하여 사용자의 질문에 답변하도록 시스템의 작동 방식을 설계합니다.

In [26]:
# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using only the following context."),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])

- 이 코드는 LangChain의 `ChatPromptTemplate`을 사용해 문맥 기반 질의응답 프롬프트를 정의합니다.
- `system` 메시지로 모델 행동 지침을 설정하고, `user` 메시지에서 동적 변수를 사용해 구체적인 입력 템플릿을 만듭니다.
- 실제 실행 시 `{context}`와 `{question}`에 데이터를 채워 넣고, 이를 통해 질문에 답변할 수 있습니다.
- 주로 **제한된 문맥에서 정보 제공**이 필요한 응용 프로그램에서 사용됩니다.

# 9. RAG 체인 구성
&emsp;이 코드는 **RAG (Retrieval-Augmented Generation)** 프로세스를 구현하기 위해 세 가지 주요 컴포넌트를 정의하고 이를 체인으로 조합하여 사용자의 질문에 대한 답변을 생성하는 방법을 보여줍니다. 아래에서 각 클래스와 구성 요소에 대해 상세히 설명하겠습니다.

In [28]:
class SimplePassThrough:
    def invoke(self, inputs, **kwargs):
        return inputs

class ContextToPrompt:
    def __init__(self, prompt_template):
        self.prompt_template = prompt_template
    
    def invoke(self, inputs):
        # 문서 내용을 텍스트로 변환
        if isinstance(inputs, list):
            context_text = "\n".join([doc.page_content for doc in inputs])
        else:
            context_text = inputs

        # 문서 내용을 텍스트로 변환
        formatted_prompt = self.prompt_template.format_messages(
            context=context_text,
            question=inputs.get("question", "")
        )
        return formatted_prompt

# Retriever를 invoke() 메서드로 래핑하는 클래스 정의
class RetrieverWrapper:
    def __init__(self, retriever):
        self.retriever = retriever

    def invoke(self, inputs):
        if isinstance(inputs, dict):
            query = inputs.get("question", "")
        else:
            query = inputs
        # 검색 수행
        response_docs = self.retriever.get_relevant_documents(query)
        return response_docs

llm_chain = LLMChain(llm=model, prompt=contextual_prompt)

# RAG 체인 설정
rag_chain_debug = {
    "context": RetrieverWrapper(retriever),
    "prompt": ContextToPrompt(contextual_prompt),
    "llm": model
}

/var/folders/68/_ywmb1rs51q00tc0pgjyrl580000gn/T/ipykernel_2261/54323330.py:37: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=model, prompt=contextual_prompt)


&emsp;이 코드는 **RAG 체인**을 구성하기 위해 각 단계별로 클래스를 정의하고 이를 조합했습니다:

- `RetrieverWrapper`: 문서 검색 담당.
- `ContextToPrompt`: 검색된 문서를 기반으로 프롬프트 생성.
- `LLMChain`: 프롬프트를 기반으로 LLM 응답 생성. 전체 체인은 문서 검색부터 답변 생성까지의 RAG 프로세스를 자동화하여 효율적으로 작동하도록 설계되었습니다.

# 10. 챗봇 구동 확인
&emsp;이 코드는 질의응답 루프를 구현합니다. 사용자는 반복적으로 질문을 입력할 수 있으며, 시스템은 RAG 체인을 통해 관련 문서를 검색하고 답변을 생성하여 출력합니다. 아래에서 각 부분을 상세히 설명합니다.

In [30]:
# 챗봇 구동
while True:
    print("========================")
    query = input("질문을 입력하세요 : ")
    
    if query.lower() in ["종료", "quit"]: # 종료하고 싶을 경우
        print("프롬프트를 종료합니다.")
        break

    # 1. Retriever로 관련 문서 검색
    response_docs = rag_chain_debug["context"].invoke({"question": query})

    # 2. 문서를 프롬프트로 변환
    prompt_messages = rag_chain_debug["prompt"].invoke({
        "context": response_docs,
        "question": query
    })

    # 3. LLM으로 응답 생성
    response = rag_chain_debug["llm"].invoke(prompt_messages)
    
    print("\n답변:")
    print(response.content)
    

질문을 입력하세요 :  종료


프롬프트를 종료합니다.


1. **사용자 입력**:
사용자가 질문을 입력하면 `query`에 저장됩니다.   
2. **문서 검색**:
`RetrieverWrapper`를 사용해 입력 질문과 관련된 문서를 검색하고 `response_docs`를 반환합니다.   
3. **프롬프트 생성**:
검색된 문서와 질문을 기반으로 프롬프트 템플릿을 채우고 `prompt_messages`를 생성합니다.   
4. **답변 생성**:
GPT-4 모델에 프롬프트를 전달하여 답변을 생성하고 `response.content`에 저장합니다.   
5. **결과 출력**:
생성된 답변을 사용자에게 출력합니다.   
6. **루프 반복**:
새로운 질문을 입력받아 위 과정을 반복합니다.